In [102]:
import random
import numpy as np
import copy


In [103]:
#[who, importance, show]
problem = [
    ["Velko", 1, 0],
    ["Andreas", 2.22, 0],
    ["Otmar",3, 0],
    ["Hugo",2,0],
    ["Christian",3,0],
    ["David",2.5,0],
    ["Thomas",1.25,0]
]

In [138]:
def objective(possible_solution):
    score = 0
    n_shown = 0
    for person in possible_solution:
        score += person[1] * person[2]
        n_shown += person[2]
    return 0.8*score - 0.2*n_shown**2

In [153]:
def genetic_solver(problem, population_size = 50, n_parents=8, mutation_probability=0.05):
    population = []
    best_fitness = -np.inf
    converged = False
    best_sample = None
    best_sample_score = -np.inf
    #make initial population and calculate fitness:
    for n_chromosome in range(population_size):
        chromosome = copy.deepcopy(problem)
        for i in range(len(chromosome)):
            chromosome[i][2] = random.randint(0, 1)
        population.append((chromosome, objective(chromosome)))

    while not converged:
        #select parents
        population.sort(key=lambda pair: pair[1], reverse=True)
        parent_population = [parent[0] for parent in population[:4]]
        children = []
        best_childs_fitness= -np.inf
        average_fitness = 0  

        while len(children) < population_size:
            parents = random.sample(parent_population, 2)
            cross_over_point = random.randint(1, len(problem[0]))
            
            chromosome = []
            for gene, p in enumerate(zip(parents[0], parents[1])):
                if gene<cross_over_point:
                    chromosome.append(p[0])
                else:
                    chromosome.append(p[1])

            if random.random() < mutation_probability:
                swap_indices = random.sample(list(range(len(chromosome))), 2)
                geneA = chromosome[swap_indices[0]][2]
                geneB = chromosome[swap_indices[1]][2]
                chromosome[swap_indices[0]][2] = geneB
                chromosome[swap_indices[1]][2] = geneA

            childs_fitness = objective(chromosome)
            if childs_fitness>best_sample_score:
                best_sample_score = childs_fitness
                best_sample = chromosome
                
            average_fitness += childs_fitness/population_size
            children.append((copy.deepcopy(chromosome), childs_fitness))

        if  average_fitness > 0.95*best_fitness:
            best_fitness = average_fitness
            population = copy.deepcopy(children)
            print("Iteration done")

        else:
            return population, best_sample

In [160]:
solution = genetic_solver(problem)
print(solution[1], objective(solution[1]))
# for s in solution:
#     print(s)





Iteration done
Iteration done
Iteration done
[['Velko', 1, 0], ['Andreas', 2.22, 1], ['Otmar', 3, 1], ['Hugo', 2, 0], ['Christian', 3, 1], ['David', 2.5, 1], ['Thomas', 1.25, 0]] 5.376
